In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
!unzip /content/utils_nlp.zip

Archive:  /content/utils_nlp.zip
  inflating: utils_nlp/__init__.py   
   creating: utils_nlp/azureml/
 extracting: utils_nlp/azureml/__init__.py  
  inflating: utils_nlp/azureml/azureml_bert_util.py  
  inflating: utils_nlp/azureml/azureml_utils.py  
  inflating: utils_nlp/azureml/README.md  
   creating: utils_nlp/common/
 extracting: utils_nlp/common/__init__.py  
  inflating: utils_nlp/common/pytorch_utils.py  
  inflating: utils_nlp/common/README.md  
  inflating: utils_nlp/common/timer.py  
   creating: utils_nlp/dataset/
  inflating: utils_nlp/dataset/__init__.py  
  inflating: utils_nlp/dataset/bbc_hindi.py  
  inflating: utils_nlp/dataset/cnndm.py  
  inflating: utils_nlp/dataset/dac.py  
  inflating: utils_nlp/dataset/data_loaders.py  
  inflating: utils_nlp/dataset/msrpc.py  
  inflating: utils_nlp/dataset/multinli.py  
  inflating: utils_nlp/dataset/ner_utils.py  
  inflating: utils_nlp/dataset/preprocess.py  
  inflating: utils_nlp/dataset/README.md  
  inflating: utils_nl

In [2]:
QUICK_RUN = True


In [3]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.models.transformers.abstractive_summarization_bertsum import (
    BertSumAbs,
    BertSumAbsProcessor,
)

from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import pprint
import scrapbook as sb

In [4]:
#pip install scrapbook

In [5]:
#pip install indic-nlp-library

In [6]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 100
if not QUICK_RUN:
    TOP_N = -1

In [7]:
train_dataset, test_dataset = CNNDMSummarizationDataset(
    top_n=TOP_N, local_cache_path=DATA_PATH, prepare_extractive=False
)

100%|██████████| 489k/489k [00:07<00:00, 62.4kKB/s]


In [8]:
len(train_dataset)


100

In [9]:
len(test_dataset)


100

In [10]:

# notebook parameters
# the cache path
CACHE_PATH = TemporaryDirectory().name

# model parameters
MODEL_NAME = "bert-base-uncased"
MAX_POS = 512
MAX_SOURCE_SEQ_LENGTH = 440
MAX_TARGET_SEQ_LENGTH = 72

# mixed precision setting. To enable mixed precision training, follow instructions in SETUP.md.
FP16 = False
if FP16:
    FP16_OPT_LEVEL = "O2"

# fine-tuning parameters
# batch size, unit is the number of tokens
BATCH_SIZE_PER_GPU = 1


# GPU used for training
NUM_GPUS = torch.cuda.device_count()
if NUM_GPUS > 0:
    BATCH_SIZE = NUM_GPUS * BATCH_SIZE_PER_GPU
else:
    BATCH_SIZE = 1


# Learning rate
LEARNING_RATE_BERT = 5e-4 / 2.0
LEARNING_RATE_DEC = 0.05 / 2.0


# How often the statistics reports show up in training, unit is step.
REPORT_EVERY = 10
SAVE_EVERY = 500

# total number of steps for training
MAX_STEPS = 1e3

if not QUICK_RUN:
    MAX_STEPS = 5e3

WARMUP_STEPS_BERT = 2000
WARMUP_STEPS_DEC = 1000

In [11]:
# processor which contains the colloate function to load the preprocessed data
processor = BertSumAbsProcessor(cache_dir=CACHE_PATH, max_src_len=MAX_SOURCE_SEQ_LENGTH, max_tgt_len=MAX_TARGET_SEQ_LENGTH)
# summarizer
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS
)

In [12]:
BATCH_SIZE_PER_GPU*NUM_GPUS


1

In [13]:
summarizer.fit(
    train_dataset,
    num_gpus=NUM_GPUS,
    batch_size=BATCH_SIZE,
    max_steps=MAX_STEPS,
    learning_rate_bert=LEARNING_RATE_BERT,
    learning_rate_dec=LEARNING_RATE_DEC,
    warmup_steps_bert=WARMUP_STEPS_BERT,
    warmup_steps_dec=WARMUP_STEPS_DEC,
    save_every=SAVE_EVERY,
    report_every=REPORT_EVERY * 5,
    fp16=FP16,
    # checkpoint="saved checkpoint path"
)

device is cuda


Iteration:  50%|█████     | 50/100 [00:12<00:12,  3.90it/s]

timestamp: 16/04/2021 06:30:18, average loss: 10.950929, time duration: 12.732745,
                            number of examples in current reporting: 50, step 50
                            out of total 1000


Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

timestamp: 16/04/2021 06:30:31, average loss: 6.530313, time duration: 12.882334,
                            number of examples in current reporting: 50, step 100
                            out of total 1000


...

timestamp: 16/04/2021 06:34:42, average loss: 1.302017, time duration: 13.338566,
                            number of examples in current reporting: 50, step 950
                            out of total 1000


Iteration:  99%|█████████▉| 99/100 [00:26<00:00,  3.74it/s]

timestamp: 16/04/2021 06:34:56, average loss: 1.628928, time duration: 13.365089,
                            number of examples in current reporting: 50, step 1000
                            out of total 1000
saving through pytorch to /tmp/tmp59jh8age/fine_tuned/bertsumabs.pt


Iteration: 100%|██████████| 100/100 [00:52<00:00,  1.90it/s]


saving through pytorch to /tmp/tmp59jh8age/fine_tuned/bertsumabs.pt


In [14]:
summarizer.save_model(MAX_STEPS, os.path.join(CACHE_PATH, "bertsumabs.pt"))


/tmp/tmp59jh8age
saving through pytorch to /tmp/tmp59jh8age/bertsumabs.pt


In [15]:
CACHE_PATH

'/tmp/tmp59jh8age'

In [16]:
checkpoint = torch.load(os.path.join(CACHE_PATH, "bertsumabs.pt"), map_location="cpu")
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS, test=True
)
summarizer.model.load_checkpoint(checkpoint['model'])

In [17]:
TEST_TOP_N = 32
if not QUICK_RUN:
    TEST_TOP_N = len(test_dataset)

if NUM_GPUS:
    BATCH_SIZE = NUM_GPUS * BATCH_SIZE_PER_GPU
else:
    BATCH_SIZE = 1
    
shortened_dataset = test_dataset.shorten(top_n=TEST_TOP_N)
src = shortened_dataset.get_source()


In [19]:
reference_summaries = [" ".join(t).rstrip("\n") for t in shortened_dataset]
generated_summaries = summarizer.predict(
    shortened_dataset, batch_size=BATCH_SIZE, num_gpus=NUM_GPUS
)
assert len(generated_summaries) == len(reference_summaries)

Generating summary:   0%|          | 0/32 [00:00<?, ?it/s]

dataset length is 32


RuntimeError: ignored

In [20]:
generated_summaries = summarizer.predict(shortened_dataset, batch_size=BATCH_SIZE, num_gpus=NUM_GPUS)


Generating summary:   0%|          | 0/32 [00:00<?, ?it/s]

dataset length is 32


TypeError: ignored

In [21]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""


In [22]:
test_dataset = SummarizationDataset(
    None, source=[source], source_preprocessing=[tokenize.sent_tokenize],
)
generated_summaries = summarizer.predict(test_dataset, batch_size=1, num_gpus=NUM_GPUS)

Generating summary:   0%|          | 0/1 [00:00<?, ?it/s]

dataset length is 1


RuntimeError: ignored

In [1]:
pip list

Package                       Version       
----------------------------- --------------
absl-py                       0.12.0        
alabaster                     0.7.12        
albumentations                0.1.12        
altair                        4.1.0         
appdirs                       1.4.4         
argon2-cffi                   20.1.0        
astor                         0.8.1         
astropy                       4.2.1         
astunparse                    1.6.3         
async-generator               1.10          
atari-py                      0.2.6         
atomicwrites                  1.4.0         
attrs                         20.3.0        
audioread                     2.1.9         
autograd                      1.3           
Babel                         2.9.0         
backcall                      0.2.0         
beautifulsoup4                4.6.3         
bleach                        3.3.0         
blis                          0.4.1         
bokeh     

In [3]:
pip install torch==1.4.0 torchvision==0.5.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 4.0MB 41.8MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
